Week Four Task
====

Team Member: 顾炜，杨光

*All the codes and files (week 1 to 4) are avalaible at [github repo](https://github.com/4everer/stormTasks)*

**任务要求**
>任务描述：

>在任务3的基础上，实现新闻类网页的分类功能。训练数据和测试数据通过公众号下载。
要求是，分类器的训练模型可以使用非Storm技术，新网页的分类必须使用Storm技术。

>最终输出：

> 1. 形成《使用Storm实现新闻分词处理报告》（包含算法说明，程序运算准确率（即分类正确的网页个数/总网页个数），程序流程图，测试说明，结果介绍）
> 2. 使用Storm实现新闻分类程序源代码；

# 程序说明

## 程序结构
(流程图)

本周项目分为两部分： 分类模型训练部分和storm部分。

首先介绍文本分类的方法和使用的模型，然后是具体的代码

### 关于文本分类

### 模型介绍
#### Naive Bayesian

#### SVM

#### xgboost

### 模型训练
分类模型训练使用python编写，有数据读入，模型训练和优化，模型存储三部分。主要使用了第三方机器学习库[sklearn](http://scikit-learn.org/)，还增加了xgboost分类算法。

(flow chart of machine learning)

**requirements**:
- sklearn
- xgboost
- matplotlib
- chardet


#### 数据读入
可以使用sklearn提供的数据，得到的数据类型是`sklearn.datasets.base.Bunch`，是一个类似于字典的类型，其中文本存在`data`中，label存在`target`中，以数字标明，而分类的名称在`target_names`中

In [1]:
from sklearn.datasets import fetch_20newsgroups
twenty_data = fetch_20newsgroups(shuffle=True, random_state=42)

all_categories = [twenty_data.target_names[t] for t in twenty_data.target]
for s in all_categories[:10]:
    print s

rec.autos
comp.sys.mac.hardware
comp.sys.mac.hardware
comp.graphics
sci.space
talk.politics.guns
sci.med
comp.sys.ibm.pc.hardware
comp.os.ms-windows.misc
comp.sys.mac.hardware


In [2]:
from pprint import pprint

target_names = twenty_data.target_names
pprint(target_names)
type(twenty_data)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


sklearn.datasets.base.Bunch

或者，从提供的数据文件读入。

其中需要注意encoding，这里使用的chardet库来判断file的编码，以下是读入的function

In [3]:
from sklearn.datasets import load_files
from os import listdir
from os.path import isfile, join
import chardet

def load_data(datadir):

    for f in listdir(datadir):
        if isfile(join(datadir, f)):
            char_encoding = chardet.detect(f)["encoding"]
            break
        else:
            continue
    
    print "============================"
    print "read files from " + datadir
    print "file encoding is " + str(char_encoding)
    
    dataset = load_files(datadir, encoding=char_encoding, decode_error="ignore")
    
    return dataset

#### 数据整理
训练数据和测试数据的建立非常重要，可以避免over fitting。针对NB和SVM，采用0.3的比例得到train和test两组数据。（以下使用sklearn提供的数据结构，与提供的数据文件内容是相同的）

In [4]:
from sklearn.cross_validation import train_test_split

twenty_train, twenty_test, y_train, y_test = train_test_split(
    twenty_data.data, twenty_data.target, test_size=0.3)

而针对文本数据，首先需要对数据进行分词计数，向量化，之后计算tf-idf，这样的得到的数据就可以用于模型训练了

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(twenty_train)
tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_counts)

#### 模型训练
使用整理好的数据，就可以进行模型的训练了

In [6]:
from sklearn.naive_bayes import MultinomialNB
export_clf_nb = MultinomialNB().fit(train_tfidf, y_train)

from sklearn.linear_model import SGDClassifier
export_clf_svm = (SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, n_iter=5, random_state=42)
                            .fit(train_tfidf, y_train))

可以用下面这个简单的例子测试一下我们刚刚训练的模型

In [7]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

for model_name, clf in zip(["nb", "svm"], [export_clf_nb, export_clf_svm]):
    predicted_new = clf.predict(X_new_tfidf)

    for doc, category in zip(docs_new, predicted_new):
        print( model_name + ' predicts: %r => %s' % (doc, target_names[category]))

nb predicts: 'God is love' => soc.religion.christian
nb predicts: 'OpenGL on the GPU is fast' => rec.autos
svm predicts: 'God is love' => soc.religion.christian
svm predicts: 'OpenGL on the GPU is fast' => rec.autos


以上过程最简明清晰可以采用Pipeline的形式，NB和SVM的训练如下：

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

def pipeline_train_classifier(classifier, x_train, y_train):
    if classifier == "nb":
        text_clf_nb = Pipeline([('vect', CountVectorizer()),
                             ('tfidf', TfidfTransformer()),
                             ('clf', MultinomialNB()),
                            ])
        return text_clf_nb.fit(x_train, y_train)
    elif classifier == "svm":
        text_clf_svm = Pipeline([('vect', CountVectorizer()),
                                 ('tfidf', TfidfTransformer()),
                                 ('clf_svm', SGDClassifier(loss='hinge', penalty='l2',
                                                  alpha=1e-4, n_iter=5, random_state=42))
                                ])
        return text_clf_svm.fit(x_train, y_train)

#### 简单评估
利用`sklearn.metrics.classsification_report`函数和之前的到的测试集，可以简单地评估模型的质量，包括准确率，召回率和F1。

- 准确率
- 召回率
- F1

In [9]:
from sklearn import metrics

def classifier_metrics(clf, x_test, y_test, target_names=None):
    predict = clf.predict(x_test)
    return metrics.classification_report(y_test, predict, target_names=target_names)

x_test = tfidf_transformer.transform(count_vect.transform(twenty_test))
for clf in [export_clf_nb, export_clf_svm]:
    print classifier_metrics(clf, x_test, y_test, target_names=target_names)

                          precision    recall  f1-score   support

             alt.atheism       0.85      0.75      0.80       139
           comp.graphics       0.86      0.75      0.80       168
 comp.os.ms-windows.misc       0.72      0.89      0.80       148
comp.sys.ibm.pc.hardware       0.70      0.81      0.75       170
   comp.sys.mac.hardware       0.91      0.86      0.88       165
          comp.windows.x       0.95      0.77      0.85       183
            misc.forsale       0.93      0.59      0.73       185
               rec.autos       0.84      0.92      0.88       178
         rec.motorcycles       0.94      0.93      0.94       181
      rec.sport.baseball       0.92      0.92      0.92       168
        rec.sport.hockey       0.87      0.99      0.93       164
               sci.crypt       0.83      0.98      0.90       197
         sci.electronics       0.95      0.77      0.85       186
                 sci.med       0.95      0.93      0.94       176
         

模型在经过优化评估之后就可以储存了（优化过程见下文），使用pickle来dump需要用到的变量

In [ ]:
import pickle

with open('export.pickle', 'wb') as f:
    pickle.dump([count_vect, tfidf_transformer, target_names, [export_clf_nb, export_clf_svm]], f)

#### xgboost
本周项目还简单测试了xgboost模型。xgboost是

In [10]:
import xgboost as xgb

dtrain = xgb.DMatrix(train_tfidf, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

param = {
    'num_class':20,
    'silent':0,
    'objective':'multi:softprob',
    'seed':42
    }

param['max_depth'] = 5
param['nthread'] = 4
param['learning_rate'] = 0.3
param['n_estimators'] = 1000
param['min_child_weight'] = 2
param['subsample'] = 0.8
param['eta'] = 1
param['gamma'] = 0.5
param['colsample_bytree'] = 0.8

evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 10
plst = param.items()

In [11]:
num_round = 10
plst = param.items()
bst = xgb.train(plst, dtrain, num_round, evallist)

[0]	eval-merror:0.392931	train-merror:0.346003
[1]	eval-merror:0.335788	train-merror:0.268089
[2]	eval-merror:0.303976	train-merror:0.231974
[3]	eval-merror:0.289249	train-merror:0.205834
[4]	eval-merror:0.273638	train-merror:0.185756
[5]	eval-merror:0.264507	train-merror:0.169971
[6]	eval-merror:0.258910	train-merror:0.155701
[7]	eval-merror:0.253314	train-merror:0.147115
[8]	eval-merror:0.249485	train-merror:0.133603
[9]	eval-merror:0.245655	train-merror:0.126657


## 分类模型的建立
### 关于文本分类

### 模型介绍
#### Naive Bayesian

#### SVM

#### xgboost

## storm topology
(结构图)

本次topology的功能是分析读取输入的一系列网址，逐个发送网址到读取url的bolt，然后获得网页内容，利用已经建立好的模型进行分类，最终汇总，输出结果。共有以下几个组件：

- `SendUrlSpout`
    在本项目中，读取还有网址的文件，然后逐行按条发送。
- `ReadFromUrlBolt`
    使用第三周的代码，接收网址的数据流，然后发出http请求，得到html源代码，解析后输出网页的标题和文字内容。
- `VectorizePredictBolt`
    读取已经训练好模型，对网页文字进行处理，然后利用模型预测分类。
- `XgbPredict`
    主要用于展示扩展性，可以在原有的模型之外，插入新的模型，进行预测。过程同样是读取训练好模型（不同的模型），对文字进行处理，最后预测分类，输出结果。
- `Aggregator`
    集中所有预测的分类，整合以待下一步处理。使用了`fieldsGrouping`按照`url`或是`title`作为key，将结果放在同一bolt汇总。 在本周的项目中，预测的结果在这一步的cleanup方法中输出到console和文件。

# 结果介绍
## 模型训练和优化
前文简单介绍了模型的训练和测试结果，而在实际运用中，需要很多的优化，包括参数的选择，特征的构造和选择等等。在不断追求更好结果的同时，也需要注意防止过拟合的发生。

### 关于数据集的优化
任意打开数据集中的一篇文章，我们就会发现，其中包含了大量的元数据，比如这一篇

```
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----
```

除了正文之外，有发信人，组织，行数等事实上相关性较小的信息，删去之后会提高模型在预测新的数据，比如网页新闻的准确性。

所以在读取数据时进行以下处理

``` python
twenty_data = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
```

### 模型参数的优化
事实上，要优化模型，应当将数据集分为三部分，**train**，**cross validation**和**test**。训练集用来训练模型，cross validation部分用来测试不同参数组合的预测效果，最终在测试数据集上检验模型的质量，这同样是为了避免过拟合。

在本项目中，使用了sklearn和xgboost自带的cross validation。下面是SVM模型参数调试部分的过程，使用了`GridSearchCV`方法。

#### grid search for SVM

In [ ]:
from __future__ import print_function

from pprint import pprint
from time import time
import logging

from sklearn.grid_search import GridSearchCV

## 参数范围的设置
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.001, 0.0001, 0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

## 数据读取，也使用了不含元数据的训练集
data = fetch_20newsgroups(subset='train')
data_no_meta = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

def gridSearch(data, name):
    print("Performing grid search for " + name)
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(data.data, data.target)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


gridSearch(data, "data with meta")
print("=============")
gridSearch(data_no_meta, "data without meta")

得到的结果如下，这样就可以使用优化过的参数进行预测了

```
Performing grid search for data with meta
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (0.001, 0.0001, 1e-05, 1e-06),
 'clf__penalty': ('l2', 'elasticnet'),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 18.6min finished
done in 1142.334s

Best score: 0.922
Best parameters set:
    clf__alpha: 0.0001
    clf__penalty: 'l2'
    vect__max_df: 1.0
    vect__ngram_range: (1, 2)

=============

Performing grid search for data without meta
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (0.001, 0.0001, 1e-05, 1e-06),
 'clf__penalty': ('l2', 'elasticnet'),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 13.0min finished
done in 802.951s

Best score: 0.752
Best parameters set:
    clf__alpha: 0.0001
    clf__penalty: 'l2'
    vect__max_df: 0.5
    vect__ngram_range: (1, 2)
```

#### xgboost优化
由于篇幅原因以及xgboost在测试中表现并不好，应该是优化不到位，模型没有建立好，所以优化过程就不介绍了。但原理上仍然是cross validation，固定大部分参数，然后测试找到一个参数的最优，再固定优化好的，测试剩余的。

## 对实际网页的测试
简单选取了一些网页如下

**sample list**


得到了下面的预测结果，依次是SVM， Naive Bayesian和Xgboost的预测